# Data retrieval Ebay Project

## Craigslist scraper

In [ ]:
#import get to call a get request on the site
from requests import get
#get the first page of the east bay housing prices
response = get('https://sfbay.craigslist.org/search/eby/apa?hasPic=1&availabilityMode=0') #get rid of those lame-o's that post a housing option without a pic using their filter
from bs4 import BeautifulSoup
html_soup = BeautifulSoup(response.text, 'html.parser')
#get the macro-container for the housing posts
posts = html_soup.find_all('li', class_= 'result-row')
print(type(posts)) #to double check that I got a ResultSet
print(len(posts)) #to double check I got 120 (elements/page)

## Ebay scraper

In [ ]:
from bs4 import BeautifulSoup
import requests

# List of item names to search on eBay
name_list = ["Ramen", "Monster Hunter World", "Adhesive page markers", "Calculator", "arduino", "gtx 1070",
             "bluetooth headphones", "coffee machine", "sweet tea", "Python textbook"]


# Returns a list of urls that search eBay for an item
def make_urls(names):
    # eBay url that can be modified to search for a specific item on eBay
    url1 = "https://www.ebay.de/sch/i.html?_odkw="
    url2 = "&_osacat=0&_from=R40&_trksid=p2045573.m570.l1313.TR12.TRC2.A0.H0.X"
    url3 = ".TRS0&_nkw="
    url4 = "&_sacat=0"
    # List of urls created
    urls = []

    for name in names:
        # Adds the name of item being searched to the end of the eBay url and appends it to the urls list
        # In order for it to work the spaces need to be replaced with a +
        urls.append(url1 + name.replace(" ", "+") + url2 + name.replace(" ", "+") + url3 + name.replace(" ", "+") + url4)
    # Returns the list of completed urls
    return urls


# Scrapes and prints the url, name, and price of the first item result listed on eBay
def ebay_scrape(urls):
    for url in urls:
        # Downloads the eBay page for processing
        res = requests.get(url, timeout=5)
        # Raises an exception error if there's an error downloading the website
        res.raise_for_status()
        # Creates a BeautifulSoup object for HTML parsing
        soup = BeautifulSoup(res.text, 'html.parser')
        # Scrapes the first listed item's name
        name = soup.find("h3", {"class": "lvtitle"}).get_text(separator=u" ")
        # Scrapes the first listed item's price
        price = soup.find("span", {"class": "bold"}).get_text()

        # Prints the url, listed item name, and the price of the item
        print(url)
        print()
        print("Item Name: " + name)
        print("Price: " + price + "\n")


# Runs the code
# 1. Make the eBay url list
# 2. Use the returned url list to search eBay and scrape and print information on each item
ebay_scrape(make_urls(name_list))

## Ebay Kleinanzeigen scraper

In [130]:
from bs4 import BeautifulSoup
import requests
import regex as re
import pandas as ps


# List of item names to search on eBay
name_list = ["auto", "wohnung"]

city_list = ["s-berlin", "s-koeln"]


# Returns a list of urls that search eBay for an item
def make_urls(names, cities):
    # eBay url that can be modified to search for a specific item on eBay
    url1 = "https://www.ebay-kleinanzeigen.de/"
    # List of urls created
    urls = []

    for name in names:
        for city in cities:
        # Adds the name of item being searched to the end of the eBay url and appends it to the urls list
        # In order for it to work the spaces need to be replaced with a +
            if (city == "s-berlin"):
                urls.append(url1 + city.replace(" ", "+") + "/anzeige:angebote/" + name.replace(" ", "+") + "/" + "k0l3331")
            elif (city == "s-koeln"):
                urls.append(url1 + city.replace(" ", "+") + "/anzeige:angebote/" + name.replace(" ", "+") + "/" + "k0l945")
            else:
                print("error")
            
    # Returns the list of completed urls
    return urls


# Scrapes and prints the url, name, and price of the first item result listed on eBay
def ebay_scrape(urls):
    title_list = []
    price_list = []
    for url in urls: 
        
        # Downloads the eBay page for processing
        res = requests.get(url, timeout=5)
        # Raises an exception error if there's an error downloading the website
        res.raise_for_status()
        # Creates a BeautifulSoup object for HTML parsing
        soup = BeautifulSoup(res.text, 'html.parser')
        names = soup.find_all("h2", {"class": "text-module-begin"})
        prices = soup.find_all("div", {"class": "aditem-details"})
        print( prices[1].get_text(separator=u" "))
#         print(names[1])
        for i in range(20):
            # Scrapes the first listed item's name
            name = names[i].get_text(separator=u" ")
            # Build lists with appended items
            title_list.append(name)
            
            # Scrapes the first listed item's price
            price = prices[i].get_text(separator=u" ")
            head, sep, tail = price.partition(' ')
            price_list.append(head)

            
            
        
#         # Prints the url, listed item name, and the price of the item
#         print()
#         print("Item Name: " + name)
#         print("Price: " + price + "\n")
    # Convert lists to pandas dataframe
#     print(titles)
#     print(prices)
    
    d = {'title': title_list, 'prices':price_list}
        
    df = pd.DataFrame(d)
        


    return df

In [131]:
df = ebay_scrape(make_urls(name_list, city_list))


 35 € 
 
                    10963 
                        Kreuzberg 


 53.900 € 
 
                    50825 
                        Bickendorf 


 850 € 
 
                    10589 
                        Charlottenburg 


  VB 
 
                    50667 
                        Köln Altstadt 



In [132]:
df

,title,prices
0,\n BMW 316i E46 AUTOMATIK KLIMA TÜV NEU MODE...,\n
1,\n Autovermietung/Auto mieten/Mietwagen/ Auto ...,\n
2,\n Volkswagen Polo 1.6 Automatik 105ps \n,\n
3,\n Renault Clio Energy TCe 120 \n,\n
4,"\n Opel Astra 1,6V Tüv bis Mai 2020 \n",\n
5,"\n Mercedes Benz A-Klasse 140, Tüv/ASU Neu bis...",\n
6,\n Vw passt tip Top Zustand Automattic . \n,\n
7,"\n Renault Modus 1,6 16V Automatik, Klima, Eve...",\n
8,\n VW Passat 2.0 FSI Highline Vollaustattung \n,\n
9,\n Auto Buch ungebunden \n,\n
